In [7]:
import json
import requests
import praw

reddit = praw.Reddit(client_id='tiBit4kBeuoOTA', 
                     client_secret='7-HPDa9q92MS0a-AnUVYb72QDzU', 
                     user_agent='TorkilSamStat', 
                     username='TorkSamStat', 
                     password='bbybbyduerinnafor')

# STATIC VARIABLES
base_url = 'https://api.pushshift.io/reddit/search/comment/'
agg_size_authors = 4
agg_size_subreddits = agg_size_authors - 2
used_authors = []
used_subreddits = []
 
# FUNCTIONS
def getKeysFromDict(data):
    users = []
    for obj in data:
        if obj['key'] not in ['AutoModerator', '[deleted]']:
            users.append(obj['key'])
    return users

def getMostActiveAuthorsInSubReddit(subreddit):
    url = base_url + '?since=100d&subreddit={}&aggs=author&agg_size={}'.format(subreddit, agg_size_authors)
    response = requests.get(url).json()
    return getKeysFromDict(response['aggs']['author'])

def getMostActiveSubRedditForAuthor(author):
    url = base_url + '?since=100d&author={}&aggs=subreddit&agg_size={}'.format(author, agg_size_subreddits)
    response = requests.get(url).json()
    return getKeysFromDict(response['aggs']['subreddit'])

In [8]:
# Get users of level 1
users_1 = getMostActiveAuthorsInSubReddit('the_donald')
users_1

['mangoroti', 'ENVYNITAZ']

In [9]:
subreddits_1 = getMostActiveSubRedditForAuthor('mangoroti')
subreddits_1

['The_Donald', 'florida']